In [1]:
# Uncomment to install necessary the library below 
# Run this notebook with python-3.10 or up
#!pip3 install bs4
!pip3 install openai


In [2]:
import openai
import json
import numpy as np

OPEN_AI_API_KEY = '<YOUR_OPEN_AI_KEY>'
openai.api_key = OPEN_AI_API_KEY 

def ask_davinci(question):
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=question,
      temperature=0.1,
      max_tokens=2000, 
      n=1
    )
    return response.choices[0].text.strip()


def ask_gpt3(question):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
        {"role": "user", "content": question}
    ]
    )
    return completion.choices[0].message.content #.strip()

def ask_character(post_list, question):
    prompt = [
        {
        "role": "system",
        "content": "You are an assistant answering questions based on what the writer YFJ wrote"
        }, 
        {
        "role": "user",
        "content": "Here are some articles the writer YFJ wrote:"
        }]
   
    for post in post_list:
        prompt.append({"role": "user", "content": f"""{post['title']}: {post['content']} """})
    
    prompt.append({
        "role": "user",
        "content": f"The writer YFJ was asked the question after the delimiter =====. Please answer the question as if you are the writer YFJ. Start the answer with the word I. Say I don't know or I don't have an opinion if there's no relevant writing from the writer.  ===== {question}"
        })

    completion = openai.ChatCompletion.create(
        model= "gpt-3.5-turbo",
        messages=prompt
    )
    return completion.choices[0].message.content #.strip()

def get_embedding(input): 
    embedding = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=input
    )
    return embedding.data[0].embedding



def cosine_similarity(vec1, vec2):
    # Compute the dot product of vec1 and vec2
    dot_product = np.dot(vec1, vec2)

    # Compute the L2 norms (Euclidean norms) of vec1 and vec2
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)

    # Compute the cosine similarity
    similarity = dot_product / (norm_vec1 * norm_vec2)

    return similarity


In [3]:
with open('greatreset-embeddings.json', 'r') as infile:
    # Write the JSON data to the file
    embedding_tuples = json.load(infile) 

print(len(embedding_tuples))
with open('greatreset.json', 'r') as infile2:
    # Write the JSON data to the file
    post_list = json.load(infile2) 
print(len(post_list))

post2content = {}
for post in post_list: 
    post2content[post['post_id']] = post 


1976
211


In [4]:
N=10 # change this number to a smaller number if you got an error for token limit overflow
#question = "What do you think of Tesla's stock price?" 
#question = "What do you think of Meta's Metaverse?"
#question = "Do you think the Fed does a good job?"
#question = "What are your stock picks for 2023?"
#question = "Do you think there will be another great depression?"
#question = "Will there be IPOs in 2023?"
#question = "What will happen to venture capital in 2023?"
question = "Do you think the Fed will lower interest rates in 2023?"
emq = get_embedding(question) 

scores = []
post_ids = []
for et in  embedding_tuples: 
    post_ids.append(et[1])
    scores.append(cosine_similarity(emq, et[0]))

# Convert the list to a numpy array


scores_array = np.array(scores)

# Get the indices that would sort the array
sorted_indices = np.argsort(scores_array)

# Get the top N indices. Note: [::-1] is used to reverse the array because argsort sorts in ascending order
top_N_indices = sorted_indices[-N:]
scores_array[top_N_indices]

#print(top_N_indices)
#print(scores_array[top_N_indices])

relevant_posts = list(set(np.array(post_ids)[top_N_indices]))
#print(relevant_posts)

input_list = []
for p in relevant_posts: 
    input_list.append(post2content[p])
result = ask_character(input_list, question)
print("============================")
print(f"Q: {question}")
print(f"YFJ Bot: {result}") 





Q: Do you think the Fed will lower interest rates in 2023?
YFJ Bot: I personally don't think the Fed will lower interest rates in 2023 unless there is another COVID-19 kind of disaster. The upward pressure of food, labor, and potentially energy prices will likely keep interest rates elevated. Based on the FOMC members' projections, the rates will continue to rise and reach around 5%, which means we will see risk-free investments yielding close to 5% in 2023. Unless the inflation rate significantly decreases or the economy experiences an unexpected downturn, I believe the Fed will keep or even raise interest rates to maintain price stability.
